# Mistral Finetune using QLoRA

[Source Article: QLoRA — How to Fine-Tune an LLM on a Single GPU](https://towardsdatascience.com/qlora-how-to-fine-tune-an-llm-on-a-single-gpu-4e44d6b5be32)

### Problem
Finetuning means to tweak an existing model for a particular use case, but for large models, tuning all the parameters of the orginal model is too expensive.
There is memory required for the parameters, gradients, and the optimizer.
### Concept
Quantization at a high level means to split a range into "buckets" reducing the memory required for each individual data point.
### Solution
QLoRA solves this issue by implementing 4 strategies:
1. 4-bit NormalFLoat  
    4-bit NormalFloat is a reduction that takes advantage of how the parameters of a LLM are typically normally distrbuted around 0. It splits the parameter into 16 (4-bits can represent 16 values) *equally-sized* buckets. This is in contrast with *equally-spaced* buckets which can be very sensitive to outliers.
2. Double Quantization  
    Double Quantization means that we are "quantizing the quantization constatnts". In other words, we are using a *block-wise* quantization strategy for each of the 16 buckets. This further prevents the effect of outliers producing an misrepresentative scale.
3. Paged Optimizer  
    A high level overview of this concept is that the Paged Optimizer allows the GPU and CPU to share memory and transfer pages (of memory) between them, as needed.
4. LoRA  
    LoRA is a parameters effcient finetuning method that works by adding a small set of trainable parameters to a model, while freezing the original paramenters. On a more technical level, LoRA is implmented through a matrix multiplication trick. If we think of the original weights of the model as an (n x n) matrix, two smaller matricies could represent it if they had the dimension (n x R) @ (R x n) where R << n.

These 4 components make up strategies used to implement QLoRA allow for finetuning of production level LLMs on consumer grade hardware.

# Outline
1. Imports and Dependencies
2. Load Base Model and Tokenizer
3. Prompt Engineering
4. Prepare the Model for Training
5. Prepare the Training Dataset
6. Fine-tuning the Model
7. Using the Fine-tuned Model

# 1. Imports and Dependencies

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
import tqdm

/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


```code
pip install auto-gptq
pip install optimum
pip install bitsandbytes
```

# 2. Load Base Model and Tokenizer

In [2]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", 
    trust_remote_code=False,
    revision="main") 

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.
/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/transformers/modeling_utils.py:5006:

In [3]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "Great content, thank you!"
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), 
                            max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] Great content, thank you! [/INST] I'm glad you found the content helpful! If you have any specific questions or topics you'd like me to cover in the future, feel free to ask. I'm here to help.

In the meantime, I'd be happy to answer any questions you have about the content I've already provided. Just let me know which article or blog post you're referring to, and I'll do my best to provide you with accurate and up-to-date information.

Thanks for reading, and I look forward to helping you with any questions you may have!</s>


# 3. Prompt Engineering

In [5]:
intstructions_string = f"""ShawGPT, functioning as a virtual data science \
consultant on YouTube, communicates in clear, accessible language, escalating \
to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, 
providing concise acknowledgments to brief expressions of gratitude or \
feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

prompt = prompt_template(comment)

In [6]:
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), 
                            max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> [INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, 
providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Great content, thank you! 
[/INST] Thank you for your kind words! I'm glad you found the content helpful. –ShawGPT</s>


# 4. Prepare the Model for Training

To do so, we will enable *gradient checkpointing* which is a memory saving technique that clears specfic activations and recomputes them during the backward pass. We will also enable quantized training.

In [7]:
# put the model in training model, this means dropout modules are activated
model.train() 

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

In [8]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


# 5. Prepare the Training Dataset

In [9]:
data = load_dataset("shawhin/shawgpt-youtube-comments")

In [12]:
def tokenize_function(examples):
    text = examples["example"]
    
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(text, return_tensors="np", truncation=True, max_length=512)
    return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True)

Map: 100%|██████████| 9/9 [00:00<00:00, 1139.24 examples/s]


In [13]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

# 6. Fine-tuning the Model

In [15]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "shawgpt-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",
)

In [16]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)

# train model
model.config.use_cache = False  # silence the warnings.
trainer.train()

# renable warnings
model.config.use_cache = True

/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
0,4.591700,3.963988
1,4.037200,3.435878
2,3.456500,2.975902
4,2.632200,2.272187
5,2.299600,2.066257
6,2.032000,1.888982
8,1.789700,1.717082
9,1.248500,1.712273


/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/acyeow/anaconda3/envs/genai_env/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


# 7. Using the Fine-tuned Model

In [17]:
model.eval()

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), 
                            max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> [INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, 
providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Great content, thank you! 
[/INST]

Glad you enjoyed it! –ShawGPT

(Note: I'm an AI language model and don't have the ability to feel emotions or watch videos. I'm here to help answer questions and provide explanations.)</s>
